# Google MT5 trial 2 by Ayush Kumar


##### in this notebook we are trying A new LLM named MT5 which is specialy designed for text summrization part


In [ ]:
# Importing the required libraries
import pandas as pd
import re
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Step 1: Load and preprocess the dataset
def preprocess_text(text):
    if isinstance(text, str):  # Check if the entry is a string
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        return text.lower()  # Convert to lowercase
    else:
        return ''  # Return an empty string if not a string (e.g., NaN)

# Load training and validation datasets
train_data = pd.read_csv('/content/drive/MyDrive/LLUMO/processed_train.csv')
val_data = pd.read_csv('/content/drive/MyDrive/LLUMO/processed_validation.csv')  # Assuming validation data is available

# Preprocess the text in 'Input' and 'Output' columns
train_data['Input'] = train_data['Input'].apply(preprocess_text)
train_data['Output'] = train_data['Output'].apply(preprocess_text)
val_data['Input'] = val_data['Input'].apply(preprocess_text)
val_data['Output'] = val_data['Output'].apply(preprocess_text)

# Step 2: Convert to HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Step 3: Load MT5 tokenizer and model
model_name = "google/mt5-small"  # You can choose mt5-small, mt5-base, mt5-large, etc.
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# Tokenization
def tokenize_function(examples):
    inputs = tokenizer(examples['Input'], truncation=True, padding='max_length', max_length=512)
    targets = tokenizer(examples['Output'], truncation=True, padding='max_length', max_length=128)  # Assuming summaries are shorter
    inputs['labels'] = targets['input_ids']
    return inputs

# Tokenize both training and validation datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/7491 [00:00<?, ? examples/s]

Map:   0%|          | 0/1634 [00:00<?, ? examples/s]

In [2]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00


In [8]:
# Step 4: Prepare the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=5e-5,
    save_steps=500,
    evaluation_strategy="steps",  # Evaluate during training
    eval_steps=500,               # Perform evaluation every 500 steps
    save_total_limit=2
)

# Step 5: Define Trainer and start fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,  # Validation dataset for evaluation
    tokenizer=tokenizer
)
for param in model.parameters():
    param.data = param.data.contiguous()

# Train the model

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_mt5')
tokenizer.save_pretrained('./fine_tuned_mt5')

print("Fine-tuning complete and model saved.")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,6.284700,4.318887
1000,3.735900,3.146604
1500,3.158700,2.906727
2000,3.275600,2.838207
2500,3.461300,2.800581
3000,2.920500,2.779632
3500,2.950700,2.755191
4000,2.831300,2.740955
4500,3.067700,2.735096
5000,3.026000,2.723583


Fine-tuning complete and model saved.


In [9]:
from transformers import pipeline, MT5ForConditionalGeneration, MT5Tokenizer

# Load the fine-tuned model and tokenizer
fine_tuned_model = MT5ForConditionalGeneration.from_pretrained('./fine_tuned_mt5')
fine_tuned_tokenizer = MT5Tokenizer.from_pretrained('./fine_tuned_mt5')

# Create a summarization pipeline
summarizer = pipeline('summarization', model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

# Define the prompt
def generate_summary(meta_review_text):
    prompt = f"""
    Summarize the following paper meta-review in a concise and informative manner:

    Meta-Review: {meta_review_text}

    Summary:
    """
    inputs = fine_tuned_tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = fine_tuned_model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
    return fine_tuned_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage with a sample meta-review
meta_review = "The paper explores new neural network architectures for image classification but lacks proper evaluation on diverse datasets, making it hard to generalize the results."
summary = generate_summary(meta_review)
print("Generated Summary:", summary)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Generated Summary: this paper presents new neural network architectures for image classification but lacks proper evaluation on diverse datasets and lacks proper evaluation on diverse datasets for image classification but lacks proper evaluation on diverse datasets and lacks proper evaluation on diverse datasets for image classification but lacks proper evaluation on diverse datasets and lacks proper evaluation on diverse datasets for image classification but lacks proper evaluation on diverse datasets and lacks proper evaluation on diverse datasets


In [12]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00


In [20]:
import pandas as pd
import re
import torch
from transformers import pipeline, MT5ForConditionalGeneration, MT5Tokenizer

# Load the fine-tuned model and tokenizer
fine_tuned_model = MT5ForConditionalGeneration.from_pretrained('./fine_tuned_mt5')
fine_tuned_tokenizer = MT5Tokenizer.from_pretrained('./fine_tuned_mt5')

# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1  # Use 0 for the first GPU or -1 for CPU

# Create a summarization pipeline with device specified
summarizer = pipeline('summarization', model=fine_tuned_model, tokenizer=fine_tuned_tokenizer, device=device)

# Define the preprocessing function
def preprocess_text(text):
    if isinstance(text, str):  # Check if the entry is a string
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        return text.lower()  # Convert to lowercase
    else:
        return ''  # Return an empty string if not a string (e.g., NaN)

# Load your test data
test_data = pd.read_csv('/content/drive/MyDrive/LLUMO/processed_test.csv')  # Update with the path to your test data

# Select the top 20 rows from the test data
test_data_top_20 = test_data.head(500).copy()  # Create a copy to avoid SettingWithCopyWarning

# Preprocess the 'Input' column in the test data
test_data_top_20.loc[:, 'Input'] = test_data_top_20['Input'].apply(preprocess_text)

# Generate summaries for the preprocessed test data rows
test_data_top_20.loc[:, 'Generated_Summary'] = test_data_top_20['Input'].apply(generate_summary)

# Load ROUGE metric after ensuring dependencies are installed
!pip install rouge_score  # Install the ROUGE score library if not already installed

from evaluate import load  # Make sure to import load from the evaluate library

# Load ROUGE metric
rouge_metric = load("rouge")

# Calculate ROUGE scores for the top 20
rouge_scores = rouge_metric.compute(
    predictions=test_data_top_20['Generated_Summary'].tolist(),
    references=test_data_top_20['Output'].tolist()
)

# Print ROUGE scores
print("ROUGE Scores:", rouge_scores)


ROUGE Scores: {'rouge1': 0.1993118095536636, 'rouge2': 0.04013189786225761, 'rougeL': 0.15473585889709227, 'rougeLsum': 0.1546420010652066}


In [16]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=09d9370c50c29b61b4ab2fe8eafee7d1aeeef483c30be7721b178f04c5f72a18
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
